In [1]:
# # relative import ../build/fast_kinematics
# import sys
# sys.path.append('../../')
import numpy as np
import time
import torch
import fast_kinematics
import pytorch_kinematics as pk

/home/xinsonglin/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
N = 4096
print(torch.__version__)

2.2.1+cu121


In [3]:
# fask kinematics model
model = fast_kinematics.FastKinematics("../kuka_iiwa.urdf", N, "lbr_iiwa_link_7")

In [4]:
# pk chain
chain = pk.build_serial_chain_from_urdf(open("../kuka_iiwa.urdf").read(), "lbr_iiwa_link_7")
d = "cuda"
dtype = torch.float32

chain = chain.to(dtype=dtype, device=d)

In [5]:
th = torch.rand(N, 7, dtype=dtype, device=d, requires_grad=True)
all_times = []
J = None
for _ in range(100):
  start_time = time.time()
  J = chain.jacobian(th)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.009592585563659668 with std: 0.007066831660406373


In [6]:
# the output of fast_kinematics is column major 1d array, so we need to reshape it
# experimenting to make sure we got it right
flat_array = torch.arange(1,43)
print(flat_array.view(-1,7,6).permute(0,2,1))

tensor([[[ 1,  7, 13, 19, 25, 31, 37],
         [ 2,  8, 14, 20, 26, 32, 38],
         [ 3,  9, 15, 21, 27, 33, 39],
         [ 4, 10, 16, 22, 28, 34, 40],
         [ 5, 11, 17, 23, 29, 35, 41],
         [ 6, 12, 18, 24, 30, 36, 42]]])


In [7]:
# flatten th
th = th.view(N*7)
all_times = []
J2 = None
for _ in range(1000):
  start_time = time.time()
  J2 = model.jacobian_mixed_frame_pytorch(th).view(-1,7,6).permute(0,2,1)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for fk: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for fk: 4.305362701416016e-06 with std: 2.0916010452697292e-05


In [8]:
assert torch.allclose(J, J2, atol=1e-5)

In [14]:
# test out pinv vs lstsq

b = torch.rand(6, 1, dtype=dtype, device=d, requires_grad=False)
x_pinv = torch.zeros(N, 7, dtype=dtype, device=d)
all_times = []
for i in range(100):
  start_time = time.time()
  # J2 is a collection of matrices but pinv seems to work on the entire batch
  J2_pinv = torch.pinverse(J2)
  if i == 0: x_pinv = torch.matmul(J2_pinv, b)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for pinv: {np.mean(all_times)} with std: {np.std(all_times)}")

# unit test correctness of pinv for an entire batch
first_pinv = J2[0].pinverse()
first_x = first_pinv @ b
assert torch.allclose(first_x, x_pinv[0], atol=1e-5)

Time taken for pinv: 0.03300390720367432 with std: 0.002120276819372533


In [15]:
all_times = []
x_lstsq = None
new_b = b.view(1,6,1).repeat(N,1,1)  # repeat the same b for all N
for i in range(100):
  start_time = time.time()
  x_lstsq = torch.linalg.lstsq(J2, new_b).solution
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for lstsq: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for lstsq: 0.024613924026489258 with std: 0.0007859006114635398


In [26]:
print(x.shape)
print(J2_lstsq.shape)
for i in range(N):
  assert torch.allclose(x[i], J2_lstsq[i], atol=10), f"i: {i}, x: {x[i]}, lstsq: {J2_lstsq[i]}"

torch.Size([4096, 7, 1])
torch.Size([4096, 7, 1])


AssertionError: i: 523, x: tensor([[-16880.7402],
        [-13899.2891],
        [ 11621.7275],
        [-34187.2422],
        [ 11638.2227],
        [-11677.9277],
        [-15916.2324]], device='cuda:0'), lstsq: tensor([[-16886.0566],
        [-13903.6670],
        [ 11630.2793],
        [-34198.0156],
        [ 11637.0029],
        [-11681.6084],
        [-15921.2510]], device='cuda:0')

In [10]:
# checking to make sure the results are the same
print(J.shape)
print(J2.shape)
assert torch.allclose(J, J2, atol=1e-5)

torch.Size([4096, 6, 7])
torch.Size([4096, 6, 7])


In [15]:
# just for fun numpy version still faster
qpos = np.random.rand(7*N)
qpos = qpos.astype(np.float32)

all_times = []
for _ in range(1000):
  start_time = time.time()
  poses = model.jacobian_mixed_frame(qpos)
  poses = poses.reshape(N,-1,6).transpose(0,2,1)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.00032803034782409666 with std: 0.0004883862096649905
